<a href="https://colab.research.google.com/github/basilsaju383/ML_project1/blob/main/auto_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer

In [4]:
df = pd.read_csv("auto_price.csv")

In [5]:
df = df.drop(columns=['Unnamed: 0', 'symboling'])

In [10]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=2)
train, val = train_test_split(train, test_size=0.2, random_state=23)

In [11]:
train_stats = train.describe()
train_stats= train_stats.transpose()
train_stats


,count,mean,std,min,25%,50%,75%,max
normalized-losses,124.0,117.919355,34.602645,65.00,93.75,115.00,130.250,256.00
make,124.0,13.612903,6.246573,1.00,9.00,13.00,20.000,22.00
fuel-type,124.0,1.104839,0.307588,1.00,1.00,1.00,1.000,2.00
aspiration,124.0,1.217742,0.414385,1.00,1.00,1.00,1.000,2.00
num-of-doors,124.0,1.620968,0.487114,1.00,1.00,2.00,2.000,2.00
body-style,124.0,2.822581,0.776153,1.00,2.00,3.00,3.000,5.00
drive-wheels,124.0,1.717742,0.534356,1.00,1.00,2.00,2.000,3.00
engine-location,124.0,1.008065,0.089803,1.00,1.00,1.00,1.000,2.00
wheel-base,124.0,98.679839,5.995527,86.60,94.50,96.50,102.400,115.60
length,124.0,173.137903,12.234754,141.10,166.30,172.40,178.925,202.60


In [12]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

In [13]:
train_x = norm(train)
test_x = norm(test)
val_x = norm(val)

In [14]:
train_x = train_x.drop(columns='price')
test_x = test_x.drop(columns='price')
val_x=val_x.drop(columns='price')
train_y = train['price']
test_y = test['price']
val_y = val['price']


In [15]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [16]:
class SimpleLinear(Layer):
    def __init__(self, units=64, activation=None):
        super(SimpleLinear, self).__init__()
        self.units = units
        self.activation=tf.keras.activations.get(activation)

In [17]:
def weightsAndBias(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="kernel",
            initial_value=w_init(shape=(input_shape[-1], self.units),
                                 dtype='float32'), trainable=True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
            initial_value=b_init(shape=(self.units,), dtype='float32'),
            trainable=True)

In [19]:
def call(self, inputs):
      return self.activation(tf.matmul(inputs, self.w) + self.b)

In [20]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(24,)),
    SimpleLinear(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    SimpleLinear(256, activation='relu'),
    SimpleLinear(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='relu')
])

In [22]:
model.compile(optimizer='adam',
              loss = rmse,
              metrics=tf.keras.metrics.RootMeanSquaredError())
h = model.fit(train_x, train_y, epochs=25)
model.evaluate(val_x, val_y)

Epoch 1/25
4/4 [==============================] - 0s 4ms/step - loss: 13692.1699 - root_mean_squared_error: 13726.4346
Epoch 2/25
4/4 [==============================] - 0s 4ms/step - loss: 13677.4463 - root_mean_squared_error: 13726.4014
Epoch 3/25
4/4 [==============================] - 0s 3ms/step - loss: 13679.0664 - root_mean_squared_error: 13726.3398
Epoch 4/25
4/4 [==============================] - 0s 4ms/step - loss: 13630.0947 - root_mean_squared_error: 13726.2686
Epoch 5/25
4/4 [==============================] - 0s 3ms/step - loss: 13602.0703 - root_mean_squared_error: 13726.4443
Epoch 6/25
4/4 [==============================] - 0s 3ms/step - loss: 13700.8477 - root_mean_squared_error: 13726.3701
Epoch 7/25
4/4 [==============================] - 0s 3ms/step - loss: 13693.6006 - root_mean_squared_error: 13726.1924
Epoch 8/25
4/4 [==============================] - 0s 4ms/step - loss: 13667.4238 - root_mean_squared_error: 13726.3252
Epoch 9/25
4/4 [==============================] 

[15784.900390625, 15784.900390625]

In [23]:
from sklearn.ensemble import IsolationForest
import numpy as np

# Assuming X is your dataset
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]])

# Create an Isolation Forest model
clf = IsolationForest(contamination=0.1)  # Contamination is the expected percentage of outliers

# Fit the model
clf.fit(X)

# Predict outliers (1 for inliers, -1 for outliers)
predictions = clf.predict(X)

print(predictions)


[-1  1  1  1  1]
